<a href="https://colab.research.google.com/github/djdongjin/IFT6135-Assignment/blob/master/A1_3_Jin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setting and Data Preprocessing

In [73]:
! nvidia-smi
! pip install mxnet-cu100

Sun Feb 10 18:43:22 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    69W / 149W |   6930MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import mxnet as mx
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import nn, loss as gloss, data as gdata

import os
import shutil
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import zipfile

In [81]:
from google.colab import drive
drive.mount('/content/gdrive')

data_path = '/content/gdrive/My\ Drive/Datasets/dogcat'
base_path = '/content/dogcat'

! rm -rf '/content/dogcat'
! cp -r $data_path /content

for f in ['trainset.zip', 'testset.zip']:
    with zipfile.ZipFile(os.path.join(base_path, f)) as z:
        z.extractall(base_path)
        
! rm -rf /content/dogcat/__MACOSX
! ls /content/dogcat

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
sample_submission.csv  testset	testset.zip  trainset  trainset.zip


In [82]:
train_valid_path = 'trainset'
test_path = 'testset'
train_path = 'train'
valid_path = 'valid'


def delete_dir(path):
    if os.path.exists(os.path.join(*path)):
        shutil.rmtree(os.path.join(*path))
        
def create_dir(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))

def devide_train_valid(base_path, train_valid_path, train_path, valid_path,
                       valid_ratio=0.2, labels=['Dog', 'Cat']):
    delete_dir([base_path, train_path])
    delete_dir([base_path, valid_path])
        
    all_files_by_label = [os.listdir(os.path.join(base_path, train_valid_path, label))
                          for label in labels]
    
    count_train = {label:0 for label in labels}
    count_valid = {label:0 for label in labels}
    
    for label, file_per_label in zip(labels, all_files_by_label):
        print('Train&Valid set, %s: %s' % (label, len(file_per_label)))
        create_dir([base_path, train_path, label])
        create_dir([base_path, valid_path, label])
        for f in file_per_label:
            if count_train[label] < len(file_per_label) * (1 - valid_ratio):
                shutil.copy(os.path.join(base_path, train_valid_path, label, f),
                            os.path.join(base_path, train_path, label, f))
                count_train[label] += 1
            else:
                shutil.copy(os.path.join(base_path, train_valid_path, label, f),
                            os.path.join(base_path, valid_path, label, f))
                count_valid[label] += 1
        print('Train set, %s: %s' % (label, count_train[label]))
        print('Valid set, %s: %s' % (label, count_valid[label]))
                
devide_train_valid(base_path, train_valid_path, train_path, valid_path)

Train&Valid set, Dog: 9999
Train set, Dog: 8000
Valid set, Dog: 1999
Train&Valid set, Cat: 9999
Train set, Cat: 8000
Valid set, Cat: 1999


In [83]:
os.path.join(base_path, train_path)

'/content/dogcat/train'

In [0]:
def load_data_iter(batch_size):
    aug_train = gdata.vision.transforms.Compose([
    #     gdata.vision.transforms.RandomResizedCrop(64, scale=(0.75, 1),
    #                                                ratio=(3.0/4.0, 4.0/3.0)),
        gdata.vision.transforms.RandomFlipLeftRight(),
        gdata.vision.transforms.RandomColorJitter(brightness=0.4, 
                                                  contrast=0.4, saturation=0.4),
        gdata.vision.transforms.RandomLighting(0.1),
        gdata.vision.transforms.ToTensor(),
    #     gdata.vision.transforms.Normalize([0.485, 0.456, 0.406], 
    #                                       [0.229, 0.224, 0.225])

    ])

    aug_test = gdata.vision.transforms.Compose([
        gdata.vision.transforms.ToTensor(),
    #     gdata.vision.transforms.Normalize([0.485, 0.456, 0.406], 
    #                                       [0.229, 0.224, 0.225])
    ])

    train_data = gdata.vision.ImageFolderDataset(
        os.path.join(base_path, train_path), flag=1)
    valid_data = gdata.vision.ImageFolderDataset(
        os.path.join(base_path, valid_path), flag=1)
    train_valid_data = gdata.vision.ImageFolderDataset(
        os.path.join(base_path, train_valid_path), flag=1)
    test_data = gdata.vision.ImageFolderDataset(
        os.path.join(base_path, test_path), flag=1)

    train_iter = gdata.DataLoader(train_data.transform_first(aug_train), batch_size,
                                  shuffle=True, last_batch='keep')
    valid_iter = gdata.DataLoader(valid_data.transform_first(aug_test), batch_size,
                                  shuffle=True, last_batch='keep')
    train_valid_iter = gdata.DataLoader(train_valid_data.transform_first(aug_train), batch_size,
                                  shuffle=True, last_batch='keep')
    test_iter  = gdata.DataLoader(test_data.transform_first(aug_test), batch_size,
                                  shuffle=False, last_batch='keep')
    
    return train_iter, valid_iter, train_valid_iter, test_iter

## Test

In [98]:
def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels, kernel_size=3,
                          padding=1, activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2, strides=2))
    return blk


def vgg(conv_arch):
    net = nn.Sequential()
    # 卷积层部分
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    # 全连接层部分
    net.add(nn.Dense(512, activation='relu'), nn.Dropout(0.1),
            nn.Dense(512, activation='relu'), nn.Dropout(0.1),
            nn.Dense(2))
    print(net)
    return net
        
lr, num_epochs, batch_size, ctx, wd= 0.05, 20, 128, mx.gpu(), 5e-4
lr_period, lr_decay= 80, 0.1

conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512))
vgg11 = vgg(conv_arch)
vgg11.initialize(init=init.Xavier(), ctx=ctx)
train_iter, valid_iter, train_valid_iter, test_iter = load_data_iter(batch_size)
train(vgg11, train_iter, valid_iter, num_epochs, lr, wd, ctx, lr_period,
      lr_decay)

Sequential(
  (0): Sequential(
    (0): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2D(None -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2D(None -> 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2D(None -> 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (4): Dense(None -> 512, Activation(relu))
  (5): Dropout(p = 0.1, axes=())
  (6): Dense(None ->

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Ignoring /content/dogcat/trainset/.DS_Store, which is not a directory.


KeyboardInterrupt: ignored

In [108]:
conv_arch = ((1, 32), (1, 64), (2, 128), (2, 256), (2, 256))
vgg11 = vgg(conv_arch)
vgg11.initialize(init=init.Xavier(), ctx=ctx)
train(vgg11, train_valid_iter, valid_iter, 30, batch_size, 5e-2, ctx)

Sequential(
  (0): Sequential(
    (0): Conv2D(None -> 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2D(None -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2D(None -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (4): Sequential(
    (0): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

## Define ResNet

In [0]:
class ResBlock(nn.Block):
    def __init__(self, n_channels, strides=1, nin=False, **kwargs):
        super(ResBlock, self).__init__(**kwargs)
        
        self.conv1 = nn.Conv2D(n_channels, kernel_size=3, padding=1, strides=strides)
        self.conv2 = nn.Conv2D(n_channels, kernel_size=3, padding=1)
        
        self.nin   = None
        if nin:
            self.nin = nn.Conv2D(n_channels, kernel_size=1, strides=strides)
            
    def forward(self, X):
        res = nd.relu(self.conv1(X))
        res = self.conv2(res)
        
        if self.nin:
            X = self.nin(X)
            
        return nd.relu(res + X)

In [0]:
class ResNet18(nn.Block):
    
    def __init__(self, n_outputs, **kwargs):
        super(ResNet18, self).__init__(**kwargs)
        
        self.net = nn.Sequential()
        self.net.add(
            nn.Conv2D(64, kernel_size=3, strides=1, padding=1),
            nn.BatchNorm(),
            nn.Activation('relu')
        )
        
        self.net.add(
            self.add_block(64, 2, nin=True),
            self.add_block(128, 2),
            self.add_block(256, 2),
            self.add_block(512, 2)
        )
        
        self.net.add(nn.GlobalAvgPool2D(), nn.Dense(n_outputs))
        
    def forward(self, X):
        return self.net(X)
        
    def add_block(self, n_channels, n_blocks, nin=False):
        net = nn.Sequential()

        if not nin:
            net.add(ResBlock(n_channels, 2, True))
            n_blocks -= 1

        for i in range(n_blocks):
            net.add(ResBlock(n_channels))

        return net

In [0]:
ctx = mx.gpu()

loss = gloss.SoftmaxCrossEntropyLoss()
resnet = ResNet18(2)
resnet.initialize(init=init.Xavier(), force_reinit=True, ctx=ctx)

def evaluate(net, test_iter, ctx):
    acc, n = 0.0, 0
    for X, y in test_iter:
        X, y = X.as_in_context(ctx), y.astype('float32').as_in_context(ctx)
        
        y_hat = net(X)
        acc += (y_hat.argmax(axis=1) == y).sum().asscalar()
        n += y.size
    return acc / n

def train(net, train_iter, test_iter, num_epochs, batch_size, lr, ctx):
    
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
    
    for i in range(1, num_epochs+1):
        
        train_acc, train_l, n, start = 0.0, 0.0, 0, time.time()
        
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.astype('float32').as_in_context(ctx)
            
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            
            l.backward()
            trainer.step(batch_size)
            train_l += l.asscalar()
            train_acc += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
            
        tm = time.time() - start
        print('epoch %s, train loss %.4f, train acc %.4f, time %.2f' % 
             (i, train_l/n, train_acc/n, tm))
        if test_iter:
            test_acc = evaluate(net, test_iter, ctx)
            print('epoch %s, test acc %.4f,' % (i, test_acc))

In [48]:
class VGGBlock(nn.Block):
    def __init__(self, num_conv, num_channel, **kwargs):
        super(VGGBlock, self).__init__(**kwargs)
        
        self.net = nn.Sequential()
        for i in range(num_conv):
            self.net.add(nn.Conv2D(num_channel, kernel_size=3, 
                              padding=1, activation='relu'))
        self.net.add(nn.MaxPool2D(pool_size=2, strides=2))
            
    def forward(self, X):
        return self.net(X)
    
class VGG(nn.Block):
    def __init__(self, conv_arch, **kwargs):
        super(VGG, self).__init__(**kwargs)
        
        self.net = nn.Sequential()
        # Conv part
        for num_conv, num_channel in conv_arch:
            self.net.add(VGGBlock(num_conv, num_channel))
        # Dense part
        self.net.add(nn.Dense(512, activation='relu'), nn.Dropout(0.1),
                     nn.Dense(512, activation='relu'), nn.Dropout(0.1),
                     nn.Dense(2))
        
    def forward(self, X):
        return self.net(X)
    
vgg11_arch = ((2, 16), (2, 32), (2, 64))
vgg11 = VGG(vgg11_arch)
vgg11.initialize(init=init.Xavier(), force_reinit=True, ctx=ctx)
vgg11

VGG(
  (net): Sequential(
    (0): VGGBlock(
      (net): Sequential(
        (0): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
      )
    )
    (1): VGGBlock(
      (net): Sequential(
        (0): Conv2D(None -> 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Conv2D(None -> 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
      )
    )
    (2): VGGBlock(
      (net): Sequential(
        (0): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
      )
    )
    (3): Dense(None -> 512, Activation(relu))
    

In [49]:
train(vgg11, train_iter, valid_iter, 10, batch_size, 1e-3, ctx)

epoch 1, train loss 0.6960593647956849, train acc 0.500125, time 15.337489604949951
epoch 1, test acc 0.5047523761880941,
epoch 2, train loss 0.6942884759902954, train acc 0.4990625, time 15.745855569839478
epoch 2, test acc 0.5130065032516258,
epoch 3, train loss 0.6939391870498657, train acc 0.497125, time 15.529584169387817
epoch 3, test acc 0.5300150075037519,
epoch 4, train loss 0.6929404029846191, train acc 0.5091875, time 15.61025619506836
epoch 4, test acc 0.5455227613806903,
epoch 5, train loss 0.6923841013908386, train acc 0.51375, time 15.56535291671753
epoch 5, test acc 0.5570285142571285,
epoch 6, train loss 0.6917909717559815, train acc 0.526125, time 15.54853105545044
epoch 6, test acc 0.5655327663831916,
epoch 7, train loss 0.6917110614776611, train acc 0.5229375, time 15.60174012184143
epoch 7, test acc 0.5732866433216608,
epoch 8, train loss 0.6910117626190185, train acc 0.534875, time 15.599258661270142
epoch 8, test acc 0.5842921460730365,
epoch 9, train loss 0.6902